In [1]:
# Role Playing
# Focus
# Tools
# Cooperation
# Guardrails
# Memory

In [2]:
# Warning control
import warnings
import sys
import os
warnings.filterwarnings('ignore')
sys.path.insert(0, '/Users/base/Desktop/G-AI')
os.environ["GROQ_API_KEY"] 

'add_new_key'

In [3]:
from crewai import Agent, Task, Crew, LLM


# Create Groq-backed CrewAI LLM
llm = LLM(
    model="groq/llama-3.1-8b-instant",
    temperature=0.7,
    api_key=os.environ["GROQ_API_KEY"] 
)

<!-- Role Playing, Focus and Cooperation 

Role Playing: Both agents have been given a role, goal and backstory.
Focus: Both agents have been prompted to get into the character of the roles they are playing.
Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together. -->

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm
)

support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

<!-- Tools, Guardrails and Memory -->


In [5]:
# Different Ways to Give Agents Tools
# Agent Level: The Agent can use the Tool(s) on any Task it performs.
# Task Level: The Agent will only use the Tool(s) when performing that specific Task.
# Note: Task Tools override the Agent Tools.

# serper_dev_tool allows the agents to search the web
# scrape_website_tool allows the agents to scrape contents on a specific website page
# website_search_tool allows the agents to perform RAG on a specific website page

from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool


In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [7]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [8]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

In [10]:
# Guardrails
# By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e04e8abf-d933-4d99-af5a-00f296ec3af9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:Error during short_term search: The OPENAI_API_KEY environment variable is not set.
Traceback (most recent call last):
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/memory/storage/rag_storage.py", line 
178, in search
    client = self._get_client()
             ^^^^^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/memory/storage/rag_storage.py", line 
108, in _get_client
    return self._client if self._client else get_rag_client()
                                             ^^^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/config/utils.py", line 70, in 
get_rag_client
    get_rag_config()
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/config/utils.py", line 50, in 
get_rag_config
    default_config = config_class()
                     ^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/pydantic/_internal/_dataclasses.py", line 121, 
in __init__
    s.__pydantic_validator__.validate_python(ArgsKwargs(args, kwargs), self_instance=s)
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/chromadb/config.py", line 59, in 
_default_embedding_function
    OpenAIEmbeddingFunction(
  File 
"/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/chromadb/utils/embedding_functions/openai_embedding_fu
nction.py", line 63, in __init__
    raise ValueError(f"The {api_key_env_var} environment variable is not set.")
ValueError: The OPENAI_API_KEY environment variable is not set.

ERROR:root:Error during entities search: The OPENAI_API_KEY environment variable is not set.
Traceback (most recent call last):
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/memory/storage/rag_storage.py", line 
178, in search
    client = self._get_client()
             ^^^^^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/memory/storage/rag_storage.py", line 
108, in _get_client
    return self._client if self._client else get_rag_client()
                                             ^^^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/config/utils.py", line 70, in 
get_rag_client
    get_rag_config()
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/config/utils.py", line 50, in 
get_rag_config
    default_config = config_class()
                     ^^^^^^^^^^^^^^
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/pydantic/_internal/_dataclasses.py", line 121, 
in __init__
    s.__pydantic_validator__.validate_python(ArgsKwargs(args, kwargs), self_instance=s)
  File "/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/crewai/rag/chromadb/config.py", line 59, in 
_default_embedding_function
    OpenAIEmbeddingFunction(
  File 
"/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/chromadb/utils/embedding_functions/openai_embedding_fu
nction.py", line 63, in __init__
    raise ValueError(f"The {api_key_env_var} environment variable is not set.")
ValueError: The OPENAI_API_KEY environment variable is not set.

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Provide clear instructions on how to add memory to a crew                                                    │
│  - Use a more descriptive title for the request                                                                 │
│  - Include any relevant documentation or references in the task                                                 │
│  - Provide clear and concise task descriptions                                                                  │
│  - Specify expected output formats                                                                              │
│  - Verify accuracy of actual output                                                                             │
│                                                                                                                 │
╰──────────────────────────────────────────── Retrieval Time: 6.07ms ─────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for     │
│  model `llama-3.1-8b-instant` in organization `org_01k9y8h1xce208nebm8d63gja1` service tier `on_demand` on      │
│  tokens per minute (TPM): Limit 6000, Used 5993, Requested 1159. Please try again in 11.52s. Need more tokens?  │
│  Upgrade to Dev Tier today at                                                                                   │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 0c66de85-0e4e-4fd6-b4d8-47b46ba627cd                                                                     │
│  Agent: Senior Support Representative                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: e04e8abf-d933-4d99-af5a-00f296ec3af9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k9y8h1xce208nebm8d63gja1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5993, Requested 1159. Please try again in 11.52s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
